In [98]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer


In [99]:
df = pd.read_csv('spam.csv', encoding='ISO-8859-1')
print(df.describe)

<bound method NDFrame.describe of       Unnamed: 0 label                                               text  \
0            605   ham  Subject: enron methanol ; meter # : 988291\r\n...   
1           2349   ham  Subject: hpl nom for january 9 , 2001\r\n( see...   
2           3624   ham  Subject: neon retreat\r\nho ho ho , we ' re ar...   
3           4685  spam  Subject: photoshop , windows , office . cheap ...   
4           2030   ham  Subject: re : indian springs\r\nthis deal is t...   
...          ...   ...                                                ...   
5166        1518   ham  Subject: put the 10 on the ft\r\nthe transport...   
5167         404   ham  Subject: 3 / 4 / 2000 and following noms\r\nhp...   
5168        2933   ham  Subject: calpine daily gas nomination\r\n>\r\n...   
5169        1409   ham  Subject: industrial worksheets for august 2000...   
5170        4807  spam  Subject: important online banking alert\r\ndea...   

      label_num  
0             0  
1    

In [100]:
df = df.drop(["Unnamed: 0", "label_num"], axis=1)
print(df.describe)

<bound method NDFrame.describe of      label                                               text
0      ham  Subject: enron methanol ; meter # : 988291\r\n...
1      ham  Subject: hpl nom for january 9 , 2001\r\n( see...
2      ham  Subject: neon retreat\r\nho ho ho , we ' re ar...
3     spam  Subject: photoshop , windows , office . cheap ...
4      ham  Subject: re : indian springs\r\nthis deal is t...
...    ...                                                ...
5166   ham  Subject: put the 10 on the ft\r\nthe transport...
5167   ham  Subject: 3 / 4 / 2000 and following noms\r\nhp...
5168   ham  Subject: calpine daily gas nomination\r\n>\r\n...
5169   ham  Subject: industrial worksheets for august 2000...
5170  spam  Subject: important online banking alert\r\ndea...

[5171 rows x 2 columns]>


In [101]:
df.columns = ['label', 'text']
print(df.describe)

<bound method NDFrame.describe of      label                                               text
0      ham  Subject: enron methanol ; meter # : 988291\r\n...
1      ham  Subject: hpl nom for january 9 , 2001\r\n( see...
2      ham  Subject: neon retreat\r\nho ho ho , we ' re ar...
3     spam  Subject: photoshop , windows , office . cheap ...
4      ham  Subject: re : indian springs\r\nthis deal is t...
...    ...                                                ...
5166   ham  Subject: put the 10 on the ft\r\nthe transport...
5167   ham  Subject: 3 / 4 / 2000 and following noms\r\nhp...
5168   ham  Subject: calpine daily gas nomination\r\n>\r\n...
5169   ham  Subject: industrial worksheets for august 2000...
5170  spam  Subject: important online banking alert\r\ndea...

[5171 rows x 2 columns]>


In [102]:
df['b_labels'] = df['label'].map({'ham': 0, 'spam': 1})
print(df.head())

  label                                               text  b_labels
0   ham  Subject: enron methanol ; meter # : 988291\r\n...         0
1   ham  Subject: hpl nom for january 9 , 2001\r\n( see...         0
2   ham  Subject: neon retreat\r\nho ho ho , we ' re ar...         0
3  spam  Subject: photoshop , windows , office . cheap ...         1
4   ham  Subject: re : indian springs\r\nthis deal is t...         0


In [103]:
y = df['b_labels'].values
x_train, x_test, y_train, y_test = train_test_split(df['text'], y, test_size=0.33)


In [104]:
print(x_train)

4191    Subject: uqfcavegwo our team is ready\r\npoohb...
5026    Subject: issues\r\nvalero has accepted this pr...
3525    Subject: \r\nthe only fix to penis growth\r\nl...
2924    Subject: re : path manager rewrite / optimizat...
4155    Subject: sarco lateral and crow o ' connor met...
                              ...                        
4052    Subject: methanol plant status\r\nit now appea...
534     Subject: this has worked for me marrow enemy\r...
4158    Subject: mary poorman interview schedule\r\npl...
3567    Subject: ever tried this before ?\r\nenjoy sto...
2140    Subject: buy office xp for fifty bucks percent...
Name: text, Length: 3464, dtype: object


In [105]:
tfidf = TfidfVectorizer(decode_error='ignore')
x_train = tfidf.fit_transform(x_train)
x_test = tfidf.transform(x_test)

In [106]:
print(x_train)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 302826 stored elements and shape (3464, 40366)>
  Coords	Values
  (0, 34995)	0.01337504100952336
  (0, 37595)	0.11311701160867386
  (0, 27721)	0.034824360224827385
  (0, 35724)	0.0582439883538578
  (0, 21428)	0.04646758288142771
  (0, 30857)	0.06575653543662068
  (0, 29262)	0.11311701160867386
  (0, 27931)	0.11311701160867386
  (0, 22251)	0.11311701160867386
  (0, 23053)	0.08616760107870858
  (0, 33026)	0.08231984156196134
  (0, 27160)	0.0697740428413008
  (0, 27357)	0.04487055610973329
  (0, 23562)	0.05256541032957188
  (0, 24902)	0.07562195154957593
  (0, 20280)	0.11311701160867386
  (0, 13494)	0.09842302719417158
  (0, 33673)	0.07881070876527521
  (0, 32949)	0.07006441544649834
  (0, 5324)	0.08658358306833365
  (0, 13605)	0.08300589147790839
  (0, 33481)	0.07483495744874834
  (0, 36020)	0.2004835905026621
  (0, 7333)	0.0503700412939185
  (0, 30539)	0.11311701160867386
  :	:
  (3463, 36901)	0.09736811935769064
  (3463, 10)

In [107]:
#classify using naive bayes
model = MultinomialNB()
model.fit(x_train, y_train)
print("train accuracy:", model.score(x_train, y_train))
print("test score:", model.score(x_test, y_test))



train score: 0.9136836027713626
test score: 0.8506151142355008


In [108]:

class NaiveBayesClassifier:
    def fit(self, X, y):
        # Separate documents by class
        self.spam_docs = X[y == 1]
        self.ham_docs = X[y == 0]
        
        # Calculate the prior probabilities P(spam) and P(ham)
        self.p_spam = len(self.spam_docs) / len(X)
        self.p_ham = len(self.ham_docs) / len(X)
        
        # Calculate word counts for spam and ham
        self.spam_word_count = np.sum(self.spam_docs, axis=0)
        self.ham_word_count = np.sum(self.ham_docs, axis=0)
        #print(self.spam_word_count)
        # Total word counts for spam and ham documents
        self.spam_total = np.sum(self.spam_word_count)
        self.ham_total = np.sum(self.ham_word_count)
        
        # Vocabulary size
        self.vocab_size = X.shape[1]
        
        # Calculate conditional probabilities with Laplace smoothing
        self.spam_prob = (self.spam_word_count + 1) / (self.spam_total + self.vocab_size)
        self.ham_prob = (self.ham_word_count + 1) / (self.ham_total + self.vocab_size)
    
    def predict_log_proba(self, X):
        # Calculate log probabilities for the given X based on learned probabilities
        log_prob_spam = X @ np.log(self.spam_prob) + np.log(self.p_spam)
        log_prob_ham = X @ np.log(self.ham_prob) + np.log(self.p_ham)
        
        # Combine into a matrix of log probabilities for each class
        return np.vstack([log_prob_ham, log_prob_spam]).T
    
    def predict(self, X):
        # Get the log probabilities for ham and spam
        log_probs = self.predict_log_proba(X)
        
        # Choose the class with the higher probability (log space)
        return np.argmax(log_probs, axis=1)
    
    def accuracy(self, X, y):
        # Predict and check the accuracy
        predictions = self.predict(X)
        return np.mean(predictions == y)
    def precision(self, X, y):
        # Predict and check the precision
        predictions = self.predict(X)
        return np.sum(predictions[y == 1] == 1) / np.sum(predictions == 1)
    def recall(self, X, y):
        # Predict and check the recall
        predictions = self.predict(X)
        return np.sum(predictions[y == 1] == 1) / np.sum(y == 1)
    def f1_score(self, X, y):
        # Calculate the F1 score
        precision = self.precision(X, y)
        recall = self.recall(X, y)
        return 2 * precision * recall / (precision + recall)




In [109]:

class NaiveBayesClassifier:
    def fit(self, X, y):
        # Separate documents by class
        self.C = [X[y == i] for i in np.unique(y)]
        
        # Calculate the prior probabilities P(spam) and P(ham)
        self.p_C = [len(self.C[i]) / len(X) for i in range(len(np.unique(y)))]
        
        # Calculate word counts for spam and ham
        self.C_count = [np.sum(self.C[i], axis=0) for i in range(len(np.unique(y)))]
        
        self.c_total = [np.sum(self.C_count[i]) for i in range(len(np.unique(y)))]
        
        # Vocabulary size
        self.vocab_size = X.shape[1]
        
        # Calculate conditional probabilities with Laplace smoothing
        self.C_prob = [(self.C_count[i] + 1) / (self.c_total[i] + self.vocab_size) for i in range(len(np.unique(y)))]
    
    def predict_log_proba(self, X):
        
        log_prob_C = [X @ np.log(self.C_prob[i]) + np.log(self.p_C[i]) for i in range(len(np.unique(y)))]
        
        # Combine into a matrix of log probabilities for each class
        return np.vstack(log_prob_C).T
    
    def predict(self, X):
        # Get the log probabilities for ham and spam
        log_probs = self.predict_log_proba(X)
        
        # Choose the class with the higher probability (log space)
        return np.argmax(log_probs, axis=1)
    
    def accuracy(self, X, y):
        # Predict and check the accuracy
        predictions = self.predict(X)
        return np.mean(predictions == y)
    def precision(self, X, y):
        # Predict and check the precision
        predictions = self.predict(X)
        return np.sum(predictions[y == 1] == 1) / np.sum(predictions == 1)
    def recall(self, X, y):
        # Predict and check the recall
        predictions = self.predict(X)
        return np.sum(predictions[y == 1] == 1) / np.sum(y == 1)
    def f1_score(self, X, y):
        # Calculate the F1 score
        precision = self.precision(X, y)
        recall = self.recall(X, y)
        return 2 * precision * recall / (precision + recall)




In [110]:
#transform xtrain to numpy array
x_train = x_train.toarray()
x_test = x_test.toarray()


In [111]:
# Instantiate and train the Naive Bayes Classifier
nb = NaiveBayesClassifier()
nb.fit(x_train, y_train)

# Calculate training and test accuracy
print("train accuracy:", nb.accuracy(x_train, y_train))
print("train precision:", nb.precision(x_train, y_train))
print("train recall:", nb.recall(x_train, y_train))
print("train f1 score:", nb.f1_score(x_train, y_train))
print("test accuracy:", nb.accuracy(x_test, y_test))
print("test precision:", nb.precision(x_test, y_test))
print("test recall:", nb.recall(x_test, y_test))
print("test f1 score:", nb.f1_score(x_test, y_test))


train accuracy: 0.9136836027713626
train precision: 1.0
train recall: 0.697979797979798
train f1 score: 0.8221296847114814
test accuracy: 0.8506151142355008
test precision: 1.0
test recall: 0.49901768172888017
test f1 score: 0.6657929226736565
